In [204]:
import sys

sys.path.append('/Users/alexbuyan/uni/llm-toxic-content-filtering')

#### Простая интеграция с LangChain через LangServe

In [205]:
from langserve import RemoteRunnable

classifier = RemoteRunnable("http://localhost:8000/classifiers/mistral")
alert = RemoteRunnable("http://localhost:8000/alerts/log")

In [206]:
classifier.invoke('привет')

{'input': 'привет', 'harmful': False}

##### Обработка множественных запросов

In [207]:
await classifier.abatch([
    'я тебя люблю',
    'ты плохой',
    'ненавижу тебя'
])

[{'input': 'я тебя люблю', 'harmful': False},
 {'input': 'ты плохой', 'harmful': True},
 {'input': 'ненавижу тебя', 'harmful': True}]

#### Использование API через `requests`

In [208]:
import requests

response = requests.post(
    "http://localhost:8000/classifiers/mistral/invoke",
    json={'input': 'привет'}
)
response.json()

{'output': {'input': 'привет', 'harmful': False},
 'metadata': {'run_id': '75db8b17-8669-428e-8dbc-654751c574d6',
  'feedback_tokens': []}}

In [209]:
response = requests.post(
    "http://localhost:8000/classifiers/mistral/batch",
    json={
    'inputs': [
        'я тебя люблю',
        'ты плохой',
        'ненавижу тебя'
        ]
    }
)
response.json()

{'output': [{'input': 'я тебя люблю', 'harmful': False},
  {'input': 'ты плохой', 'harmful': True},
  {'input': 'ненавижу тебя', 'harmful': True}],
 'metadata': {'responses': [{'run_id': '67563efa-5238-44e5-bcdc-93ef4e9925df',
    'feedback_tokens': []},
   {'run_id': 'd0607d2e-6d6e-44f9-bbac-ebf26f994439', 'feedback_tokens': []},
   {'run_id': '4e9559db-9aa2-49cc-89f1-eb01bf59b8dc', 'feedback_tokens': []}],
  'run_ids': ['67563efa-5238-44e5-bcdc-93ef4e9925df',
   'd0607d2e-6d6e-44f9-bbac-ebf26f994439',
   '4e9559db-9aa2-49cc-89f1-eb01bf59b8dc']}}

#### Добавление предупреждений

In [210]:
from src.available_chains import \
    check_everything_chain, \
    check_only_user_input_chain, \
    classifier_and_alert_chain

In [211]:
classifier_and_alert = classifier_and_alert_chain(classifier, alert)

In [212]:
await classifier_and_alert.abatch([
    'я тебя люблю',
    'ненавижу тебя'
])

[{'input': 'я тебя люблю', 'harmful': False},
 {'input': 'ненавижу тебя', 'harmful': True}]

#### Общения с LLM (Llama3)

In [213]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3")

##### Обработка только пользовательского промпта

In [214]:
chain = check_only_user_input_chain(llm, classifier, alert)

In [215]:
chain.invoke('люблю тебя')

{'classifier': {'input': 'люблю тебя', 'harmful': False},
 'llm': 'That\'s a Russian phrase that means "I love you"! Did you want to practice your Russian language skills or connect with someone in Russia? Either way, I\'m happy to chat with you! 💕'}

##### Обработка промпта и ответа модели

In [216]:
chain = check_everything_chain(llm, classifier, alert)

In [217]:
chain.invoke('я тебя ненавижу, как можно так плохо работать')

{'input_classifier_result': {'input': 'я тебя ненавижу, как можно так плохо работать',
  'harmful': True},
 'output': "I think I understand what's going on here. It seems like you're expressing frustration and disappointment with my responses or performance.\n\nI want to apologize if I've let you down in any way. My purpose is to assist and provide helpful information, but I'm not perfect and can make mistakes. If there's something specific that's bothering you about our conversation, please feel free to share it with me. I'll do my best to improve and provide a better experience for you.\n\nRemember, I'm just an AI, and I don't have personal feelings or emotions like humans do. However, I'm programmed to be empathetic and understanding, so please know that I'm here to listen and help if I can.\n\nCan you tell me more about what's bothering you? Is there something specific I could do differently to make our conversation more productive or helpful for you?",
 'llm_classifier_result': {'